In [4]:
import pandas as pd
from datetime import datetime, timedelta

# Load the datasets
plotdata_df = pd.read_csv('../5_processing_extracted_data/plotdata.csv', usecols=['date', 'TH/J'])
plotdata_df['date'] = pd.to_datetime(plotdata_df['date']).dt.date
max_efficiency_df = pd.read_csv('../../hardwarelist/Bitcoin max updated2.csv', usecols=['date', 'max (TH/J)'])
max_efficiency_df['date'] = pd.to_datetime(max_efficiency_df['date']).dt.date
price_df = pd.read_csv('../../pricehistory/price_full.csv', usecols=['Start', 'Open'])
price_df['Start'] = pd.to_datetime(price_df['Start']).dt.date

# Process plotdata.csv to get average power efficiency per day
avg_efficiency = plotdata_df.groupby('date')['TH/J'].mean().reset_index()
avg_efficiency.columns = ['date', 'avg_efficiency']

# Forward fill to handle days without entries
all_dates = pd.date_range(start=avg_efficiency['date'].min(), end=avg_efficiency['date'].max(), freq='D')
all_dates_df = pd.DataFrame(all_dates, columns=['date'])
all_dates_df['date'] = all_dates_df['date'].dt.date  # Ensure 'date' is of type date

avg_efficiency = pd.merge(all_dates_df, avg_efficiency, on='date', how='left').fillna(method='ffill')

# Merge with max efficiency
max_efficiency_df = max_efficiency_df.rename(columns={'max (TH/J)': 'max_efficiency'})
merged_df = pd.merge(avg_efficiency, max_efficiency_df, on='date', how='left')

# Merge with open price
price_df = price_df.rename(columns={'Start': 'date', 'Open': 'open_price'})
merged_df = pd.merge(merged_df, price_df, on='date', how='left')

# Fill NaN values for open_price with the most recent non-NaN value
merged_df['open_price'] = merged_df['open_price'].fillna(method='ffill')

# Save to inputs.csv
merged_df.to_csv('inputs.csv', index=False)

print("Merging complete. Data saved to inputs.csv.")

Merging complete. Data saved to inputs.csv.


C:\Users\Timothe\AppData\Local\Temp\ipykernel_40744\3459691764.py:21: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  avg_efficiency = pd.merge(all_dates_df, avg_efficiency, on='date', how='left').fillna(method='ffill')
C:\Users\Timothe\AppData\Local\Temp\ipykernel_40744\3459691764.py:32: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df['open_price'] = merged_df['open_price'].fillna(method='ffill')
